# Calibration of potential evapotranspiration methods
*M. Vremec, R.A. Collenteur University of Graz, 2021*

In this notebook it is shown how to calibrate various (potential) evapotranspiration (PET) equations, using a linear regression relationship between daily potential evapotranspiration obtained with the FAO-56 equation and daily PET estimates obtained with the alternative methods. This notebook requires Scipy and SkLearn python packages to run.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from sklearn.metrics import mean_squared_error

import pyet as pyet

## 1. Load KNMI Data

In [ ]:
data = pd.read_csv("data/example_3/etmgeg_260.txt", skiprows=46, delimiter=",", 
                   skipinitialspace=True, index_col="YYYYMMDD", parse_dates=True).loc["2018",:]
data.head()

## 2.Estimation of potential evapotranspiration

In [ ]:
# define meteorological variables needed for PE estimation
meteo = pd.DataFrame({"tmean":data.TG/10, "tmax":data.TX/10, "tmin":data.TN/10, "rh":data.UG, "wind":data.FG/10, "rs":data.Q/100})
tmean, tmax, tmin, rh, wind, rs = [meteo[col] for col in meteo.columns]
pressure =  data.PG / 100 # to kPa
wind = data.FG/10  # to m/s
lat = 0.91  # [rad]
elevation = 4 

pet_fao56 = pyet.pm_fao56(tmean, wind, rs=rs, elevation=elevation, lat=lat, 
                         tmax=tmax, tmin=tmin, rh=rh)  # FAO-56 method
pet_romanenko = pyet.romanenko(tmean, rh)  # Romanenko method
pet_abtew = pyet.abtew(tmean, rs)  # Abtew method

The model performance of the Abtew and Romanenko method will be evaluated using the root mean square error (RMSE), as implemented in SkLearn’s mean_squared_error method.

In [ ]:
pet_fao56.plot()
pet_romanenko.plot()
pet_abtew.plot()
plt.ylabel("PE [mm/day]")
print("RMSE(Romanenko) = {} mm/d".format(mean_squared_error(pet_fao56, pet_romanenko, squared=False)))
print("RMSE(Abtew) = {} mm/d".format(mean_squared_error(pet_fao56, pet_abtew, squared=False)))

## 3. Calibration of different PE equations

The least squares approach is applied to estimate the parameters in the PE equations, by minimizing the sum of the residuals between the simulated (Abtew and Romanenko) and observed (FAO-56) data. The minimization of the objective function is done using the Trust Region Reflective algorithm, as implemented in Scipy’s least squares method.

### 3.1 Romanenko method

In [ ]:
# Define function for computing residuals
def cal_romanenko(k, obs):
    return pyet.romanenko(tmean, rh, k)-obs

In [ ]:
# estimate k in the Romanenko method
x0 = 4.5  # initial estimate of parameter
res_1 = least_squares(cal_romanenko, x0, args=[pet_fao56])
res_1.x

In [ ]:
# Compute RMSE using the calibrated value of k
pet_romanenko_cal = pyet.romanenko(tmean, rh, k=res_1.x)
print("RMSE(Romanenko) = {} mm/d".format(mean_squared_error(pet_fao56, pet_romanenko_cal, squared=False)))

RMSE (calibrated) = 0.546 < RMSE (uncalibrated) = 0.694

### 3.2 Abtew method

In [ ]:
# Define function for computing residuals and initial estimate of parameters
def cal_abtew(k,obs):
    return pyet.abtew(tmean, rs, k)-obs
x0 = 0.53

In [ ]:
# estimate k in the Romanenko method
res_2 = least_squares(cal_abtew, x0, args=[pet_fao56])
res_2.x

In [ ]:
pet_abtew_cali = pyet.abtew(tmean, rs, res_2.x)
print("RMSE(Abtew) = {} mm/d".format(mean_squared_error(pet_fao56, pet_abtew_cali, squared=False)))

RMSE (calibrated) = 0.613 < RMSE (uncalibrated) = 0.741

In [ ]:
pet_fao56.plot()
pet_romanenko_cal.plot()
pet_abtew_cali.plot()
plt.ylabel("PET [mm/d]");